In [1]:
using Pkg

Pkg.activate("Agate")
Pkg.status()

  Activating project at `~/Documents/Projects/Agate.jl`


Project Agate v0.0.1
Status `~/Documents/Projects/Agate.jl/Project.toml`
  [a49af516] OceanBioME v0.11.1
  [9e8cae18] Oceananigans v0.91.13


In [2]:
using Agate

# The simplest case

You have your own set of functions which you write out fully where the only variables are other tracers (or auxiliary fields) and everything else is a constant.

The user needs to provide a dictionary of tracer functions as Julia expression objects (they have to be defined inside `:(...)`. One can also optionally specify auxiliary fields as a vector.

In [3]:
tracers = Dict(
    "N" => :(N + Z + D + P),
    "P" => :(0.5 * PAR),
    "D" => :(0.1 * D),
    "Z" => :(P-1)
)

aux_field_vars = [:PAR,]

1-element Vector{Symbol}:
 :PAR

In [4]:
model = construct_bgc_model(tracers, aux_field_vars)

Agate.Models.Dynamic.DynamicBGC()

In [5]:
model(Val(:N), 0, 0, 0, 0, 1, 1, 1, 1, 1)

4

# Using Agate functions

In a better world, we make use of existing functions in the Agate library. 

In this case, the Dynamic model submodule needs to have visibility of all functions, otherwise it won't work. This seems fine.

We also need a way for the user to specify the constant terms and currently they have to do this manually (even when these repeat) - this should be fixed.

In [6]:
tracers = Dict(
    "N" => :(phytoplankton_metabolic_loss(P, lᵖⁿ=0.066) 
    + zooplankton_metabolic_loss(Z, lᶻⁿ=0.0102) 
    + remineralization(D, rᵈⁿ=0.1213) 
    - phytoplankton_growth(N, PAR, P, μ₀=0.698, kₙ=2.3868, α=0.1953)),

    "D" => :(phytoplankton_mortality_loss(P, lᵖᵈ=0.0101)
    + zooplankton_assimilation_loss(P, Z, β=0.9116, gₘₐₓ=2.1522, kₚ=0.5573) 
    + zooplankton_mortality_loss(Z, lᶻᵈ=0.0102) 
    - remineralization(D, rᵈⁿ=0.1213) ),
    
    "P" => :(phytoplankton_growth(N, PAR, P, μ₀=0.698, kₙ=2.3868, α=0.1953) 
    - phytoplankton_grazing_loss(P, Z, gₘₐₓ=2.1522, kₚ=0.5573) 
    - phytoplankton_metabolic_loss(P, lᵖⁿ=0.066) 
    - phytoplankton_mortality_loss(P, lᵖᵈ=0.0101)),
    
    "Z" => :(zooplankton_grazing_gain(P, Z, β=0.9116, gₘₐₓ=2.1522, kₚ=0.5573) 
    - zooplankton_metabolic_loss(Z, lᶻⁿ=0.0102)  
    - zooplankton_mortality_loss(Z, lᶻᵈ=0.3395) )
    )

aux_field_vars = [:PAR,]

1-element Vector{Symbol}:
 :PAR

Then one creates an Oceananigans BGC model by calling `construct_bgc_model`:

In [7]:
model2 = construct_bgc_model(tracers, aux_field_vars)

Agate.Models.Dynamic.DynamicBGC()

The tracer methods can then be called as expected, for example

In [23]:
# the first 4 values should not matter - those are the x,y,z,t coordinates followed by the tracers
model2(Val(:N), 0, 0, 0, 0, 1, 1, 1, 1, 1), model2(Val(:D), 0, 0, 0, 0, 1, 1, 1, 1, 1)

(0.14196773816764147, 0.044167790137168664)

### TEST

For comparison, we define a function for tracer N same as above and with all the same parameter values... we should get the same result....

In [9]:
pwd()

"/Users/rjersakova/Documents/Projects/Agate.jl/notebooks"

In [10]:
# all the functions are defined here
include("../src/Models/Library.jl")

zooplankton_assimilation_loss (generic function with 1 method)

In [20]:
# just copy pasting expression from the top....
function N(D, N, P, Z, PAR)
    return phytoplankton_metabolic_loss(P, lᵖⁿ=0.066)  + zooplankton_metabolic_loss(Z, lᶻⁿ=0.0102) + remineralization(D, rᵈⁿ=0.1213)  - phytoplankton_growth(N, PAR, P, μ₀=0.698, kₙ=2.3868, α=0.1953)
end

function D(D, N, P, Z, PAR)
    phytoplankton_mortality_loss(P, lᵖᵈ=0.0101) + zooplankton_assimilation_loss(P, Z, β=0.9116, gₘₐₓ=2.1522, kₚ=0.5573)  + zooplankton_mortality_loss(Z, lᶻᵈ=0.0102)   - remineralization(D, rᵈⁿ=0.1213)
end

D (generic function with 1 method)

In [21]:
N(1, 1, 1, 1, 1), D(1, 1, 1, 1, 1)

(0.066, -0.1213)

## Latex

Example of turning functions into latex

In [13]:
# return latex 
using Latexify
latexify(:(x + y)) # |> print

L"$x + y$"